In [ ]:
from local.torch_basics import *
from local.test import *
from local.core import *
#from local.layers import *
from local.data.all import *
#from local.optimizer import *
#from local.learner import *
#from local.metrics import *
#from local.callback.all import *
from local.vision.core import *
from local.vision.augment import *

In [ ]:
path = untar_data(URLs.CAMVID)

In [ ]:
valid_fnames = (path/'valid.txt').read().split('\n')

In [ ]:
def ListSplitter(valid_items):
    def _inner(items):
        val_mask = tensor([o.name in valid_items for o in items])
        return [~val_mask,val_mask]
    return _inner

In [ ]:
camvid = DataBlock(types=(PILImage, PILMask),
                   get_items=get_image_files,
                   splitter=ListSplitter(valid_fnames),
                   get_y=lambda o: path/'labels'/f'{o.stem}_P{o.suffix}')

In [ ]:
def rm_class(x): return torch.Tensor._make_subclass(Tensor, tensor(x))
rm_class.order=999

In [ ]:
sz = tuple(int(o) for o in np.array([360,480]))

In [ ]:
dbunch = camvid.databunch(path/"images", bs=8, item_tfms=Resize(sz), 
                          batch_tfms=[*aug_transforms(size=(360,480)), Normalize(*imagenet_stats), rm_class])

In [ ]:
len(dbunch.train_ds.tls[0].items),len(dbunch.valid_ds.tls[0].items)

(600, 101)

In [ ]:
#dbunch.show_batch(max_n=4, vmin=1, vmax=30, figsize=(20,14))

In [ ]:
codes = np.loadtxt(path/'codes.txt', dtype=str)
dbunch.vocab = codes

In [ ]:
name2id = {v:k for k,v in enumerate(codes)}
void_code = name2id['Void']

def acc_camvid(input, target):
    target = target.squeeze(1)
    mask = target != void_code
    return (input.argmax(dim=1)[mask]==target[mask]).float().mean()

In [ ]:
#opt_func = partial(Adam, lr=3e-3, wd=0.01)#, eps=1e-8)

In [ ]:
#learn = unet_learner(dbunch, resnet34, loss_func=CrossEntropyLossFlat(axis=1), opt_func=opt_func, path=path, metrics=acc_camvid,
#                     config = unet_config(norm_type=None), wd_bn_bias=True)

In [ ]:
from fastai.vision import *

In [ ]:
dbunch.c = len(codes)
dbunch.path = path
dbunch.empty_val=False
learn = unet_learner(dbunch, models.resnet34, loss_func=CrossEntropyFlat(axis=1), path=path, wd=1e-2, metrics=acc_camvid)

In [ ]:
lr= 3e-3
learn.freeze()

In [ ]:
learn.fit_one_cycle(10, slice(lr), pct_start=0.9)

epoch,train_loss,valid_loss,acc_camvid,time
0,1.299427,0.903493,0.794205,00:49
1,0.906711,0.652964,0.821146,00:36
2,0.715029,0.549639,0.848226,00:36
3,0.732738,0.674755,0.832838,00:36
4,0.645157,0.470718,0.871823,00:36
5,0.584654,0.461729,0.881502,00:37
6,0.584426,0.441123,0.882653,00:37
7,0.583593,0.451077,0.874695,00:37
8,0.564715,0.426055,0.883774,00:37
9,0.486960,0.360778,0.900210,00:37


In [ ]:
learn.save('stage-1')

In [ ]:
#learn.show_results(max_n=4, vmin=1, vmax=30)

In [ ]:
learn.load('stage-1')
learn.unfreeze()

In [ ]:
#learn.opt.clear_state()

In [ ]:
lrs = slice(lr/400,lr/4)

In [ ]:
learn.fit_one_cycle(12, lrs, pct_start=0.8)

epoch,train_loss,valid_loss,acc_camvid,time
0,0.415595,0.346189,0.900587,00:39
1,0.405366,0.341612,0.903008,00:39
2,0.395798,0.344117,0.901581,00:39
3,0.382426,0.322022,0.905388,00:39
4,0.377685,0.321676,0.910287,00:40
5,0.369895,0.334568,0.904465,00:40
6,0.359500,0.343446,0.903857,00:39
7,0.351581,0.305634,0.915762,00:39
8,0.341745,0.304973,0.915286,00:39
9,0.338542,0.284871,0.920472,00:39


In [ ]:
#learn.show_results(max_n=4, vmin=1, vmax=30, figsize=(15,6))